In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score # MSE y R2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from prettytable import PrettyTable


In [291]:
df = pd.read_csv('housing_Tarea2.csv')
df

,house_type,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,verificationDate,description,SecurityDeposit,Status
0,1 RK Studio Apartment,400 sq ft,Kalkaji,Delhi,28.545561,77.254349,22000,INR,1.0,NaN,NaN,NaN,Posted a day ago,"Fully furnished, loaded with amenities & gadge...",No Deposit,Furnished
1,1 RK Studio Apartment,400 sq ft,Mansarover Garden,Delhi,28.643259,77.132828,20000,INR,1.0,NaN,NaN,NaN,Posted 9 days ago,Here is an excellent 1 BHK Independent Floor a...,No Deposit,Furnished
2,2 BHK Independent Floor,500 sq ft,Uttam Nagar,Delhi,28.618677,77.053352,8500,INR,1.0,NaN,NaN,NaN,Posted 12 days ago,"Zero Brokerage.\r\n\r\n2 Room set, Govt bijali...",No Deposit,Semi-Furnished
3,3 BHK Independent House,"1,020 sq ft",Model Town,Delhi,28.712898,77.180000,48000,INR,3.0,NaN,NaN,NaN,Posted a year ago,Itâs a 3 bhk independent house situated in M...,No Deposit,Furnished
4,2 BHK Apartment,810 sq ft,Sector 13 Rohini,Delhi,28.723539,77.131424,20000,INR,2.0,NaN,NaN,NaN,Posted a year ago,Well designed 2 bhk multistorey apartment is a...,No Deposit,Unfurnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4 BHK Villa,"5,896 sq ft",Sunder Nagar,Delhi,28.618437,76.961784,1022001,INR,4.0,2.0,NaN,NaN,Posted 2 months ago,Its four bhk villa in the super location of De...,"40,10,102",Unfurnished
4996,5 BHK Independent House,"6,521 sq ft",Sunder Nagar,Delhi,28.618437,76.961784,1549181,INR,4.0,2.0,NaN,NaN,Posted 2 months ago,A 5 bhk property is available for rent in Sund...,"54,01,015",Unfurnished
4997,3 BHK Independent Floor,"1,855 sq ft",New Friends Colony,Delhi,28.567051,77.273560,301012,INR,3.0,2.0,NaN,NaN,Posted 2 months ago,Its three bhk builder floor in the super locat...,"18,18,181",Unfurnished
4998,3 BHK Independent Floor,"2,856 sq ft",New Friends Colony,Delhi,28.567051,77.273560,301011,INR,3.0,2.0,NaN,NaN,Posted 2 months ago,Its three bhk builder floor in the super locat...,"10,10,110",Unfurnished


In [272]:
#Eliminable
df["description"].iloc[0] # bag of words, TF-IDF, word embeddings
type(df["priceSqFt"].iloc[0]) 
df["SecurityDeposit"].value_counts()
df["isNegotiable"].value_counts()


isNegotiable
Negotiable    179
Name: count, dtype: int64

## Limpieza de datos


In [292]:
# Limpieza de datos para la columna house_type
# "mujer" "hombre" <- 0 1 <- encoding, con el tipo de Label
df["numRooms"] = df["house_type"].str[0].astype(int)
# 3 BHK <- 3 Bedroom, 1 Hall, 1 Kitchen
#df["house_type"].value_counts()
df["rooms"] = df["house_type"].str.split(" ").str[1]
df["housing_type"] = df["house_type"].str.split(" ").str[2:].str.join(" ")
#df["isNegotiable"].value_counts() 
#df["isNegotiable"] = df["isNegotiable"].str.split(" ").str[2]


# Encoding características categóricas
encoder = LabelEncoder()
df["rooms"] = encoder.fit_transform(df["rooms"]).astype(int)
df["housing_type"] = encoder.fit_transform(df["housing_type"]).astype(int)
df["isNegotiable"] = encoder.fit_transform(df["isNegotiable"]).astype(int)
df["isNegotiable"] = 1 - df["isNegotiable"] # 1 si es negociable, 0 si es nan
df["Status"] = encoder.fit_transform(df["Status"]).astype(int)
df.drop(columns=["house_type"], inplace=True)

In [293]:
# Limpieza columna house_size
df["house_size"] = df["house_size"].str.replace(",","") 
df["house_size"] = df["house_size"].str.split(" ").str[0].astype(int)

In [294]:
#limpieza security deposit
df['SecurityDeposit'] = (df['SecurityDeposit'].astype(str).str.strip().str.replace(',', '').str.replace('No Deposit', '0'))
df['SecurityDeposit'] = df['SecurityDeposit'].fillna(0).astype(int)


In [295]:
df["location"] = encoder.fit_transform(df["location"]).astype(int)
df["city"] = encoder.fit_transform(df["city"]).astype(int)
df["currency"] = encoder.fit_transform(df["currency"]).astype(int)
df = df.fillna(0) 
df


,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,verificationDate,description,SecurityDeposit,Status,numRooms,rooms,housing_type
0,400,88,0,28.545561,77.254349,22000,0,1.0,0.0,0,0.0,Posted a day ago,"Fully furnished, loaded with amenities & gadge...",0,0,1,1,3
1,400,124,0,28.643259,77.132828,20000,0,1.0,0.0,0,0.0,Posted 9 days ago,Here is an excellent 1 BHK Independent Floor a...,0,0,1,1,3
2,500,259,0,28.618677,77.053352,8500,0,1.0,0.0,0,0.0,Posted 12 days ago,"Zero Brokerage.\r\n\r\n2 Room set, Govt bijali...",0,1,2,0,1
3,1020,133,0,28.712898,77.180000,48000,0,3.0,0.0,0,0.0,Posted a year ago,Itâs a 3 bhk independent house situated in M...,0,0,3,0,2
4,810,201,0,28.723539,77.131424,20000,0,2.0,0.0,0,0.0,Posted a year ago,Well designed 2 bhk multistorey apartment is a...,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5896,249,0,28.618437,76.961784,1022001,0,4.0,2.0,0,0.0,Posted 2 months ago,Its four bhk villa in the super location of De...,4010102,2,4,0,4
4996,6521,249,0,28.618437,76.961784,1549181,0,4.0,2.0,0,0.0,Posted 2 months ago,A 5 bhk property is available for rent in Sund...,5401015,2,5,0,2
4997,1855,146,0,28.567051,77.273560,301012,0,3.0,2.0,0,0.0,Posted 2 months ago,Its three bhk builder floor in the super locat...,1818181,2,3,0,1
4998,2856,146,0,28.567051,77.273560,301011,0,3.0,2.0,0,0.0,Posted 2 months ago,Its three bhk builder floor in the super locat...,1010110,2,3,0,1


In [296]:
verif_map = {
    'day' : 1, 'days' : 1,
    "week" : 7, "weeks" : 7, 
    "month" : 30, "months" : 30,
    "year" : 365, "year" : 365
}
df[["cant_veces_str", "mult"]] = df["verificationDate"].str.extract(r'Posted\s+(\d+|a|an)\s+(\w+)')
df['cant_veces'] = df['cant_veces_str'].replace({'a': 1, 'an': 1}).astype(int)
df["days_since_verif"] = df["cant_veces"] * df["mult"].map(verif_map)
df.drop(columns=["verificationDate", "cant_veces_str", "mult", "cant_veces","description"], inplace=True)

df["days_since_verif"] = df["days_since_verif"].fillna(0).astype(int)

df_baseline = df.copy()
df_baseline

,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
0,400,88,0,28.545561,77.254349,22000,0,1.0,0.0,0,0.0,0,0,1,1,3,1
1,400,124,0,28.643259,77.132828,20000,0,1.0,0.0,0,0.0,0,0,1,1,3,9
2,500,259,0,28.618677,77.053352,8500,0,1.0,0.0,0,0.0,0,1,2,0,1,12
3,1020,133,0,28.712898,77.180000,48000,0,3.0,0.0,0,0.0,0,0,3,0,2,365
4,810,201,0,28.723539,77.131424,20000,0,2.0,0.0,0,0.0,0,2,2,0,0,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5896,249,0,28.618437,76.961784,1022001,0,4.0,2.0,0,0.0,4010102,2,4,0,4,60
4996,6521,249,0,28.618437,76.961784,1549181,0,4.0,2.0,0,0.0,5401015,2,5,0,2,60
4997,1855,146,0,28.567051,77.273560,301012,0,3.0,2.0,0,0.0,1818181,2,3,0,1,60
4998,2856,146,0,28.567051,77.273560,301011,0,3.0,2.0,0,0.0,1010110,2,3,0,1,60


In [297]:
df_baseline.describe()

,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
count,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5.000000e+03,5000.0,5000.000000,5000.000000,5000.00000,5000.0,5.000000e+03,5000.000000,5000.00000,5000.00000,5000.000000,5000.000000
mean,2982.885400,136.314400,0.0,28.578012,77.174499,2.221738e+05,0.0,2.904000,1.069800,0.03580,0.0,6.569098e+05,1.321200,3.09780,0.02760,1.210400,127.395400
std,2168.663368,79.323699,0.0,0.190186,0.115636,2.739843e+05,0.0,1.104458,1.053731,0.18581,0.0,9.879070e+05,0.715914,1.14162,0.16384,1.045347,136.765496
min,150.000000,0.000000,0.0,20.011379,72.771332,3.000000e+03,0.0,0.000000,0.000000,0.00000,0.0,0.000000e+00,0.000000,1.00000,0.00000,0.000000,0.000000
25%,1100.000000,65.000000,0.0,28.544489,77.138248,2.950000e+04,0.0,2.000000,0.000000,0.00000,0.0,0.000000e+00,1.000000,2.00000,0.00000,1.000000,14.000000
50%,2500.000000,141.000000,0.0,28.569295,77.196472,1.250000e+05,0.0,3.000000,1.000000,0.00000,0.0,3.600000e+04,1.000000,3.00000,0.00000,1.000000,60.000000
75%,5896.000000,195.000000,0.0,28.618687,77.228950,3.011020e+05,0.0,4.000000,2.000000,0.00000,0.0,1.012010e+06,2.000000,4.00000,0.00000,1.000000,180.000000
max,14521.000000,287.000000,0.0,28.805466,80.361313,3.010101e+06,0.0,10.000000,8.000000,1.00000,0.0,1.140101e+07,2.000000,9.00000,1.00000,5.000000,365.000000


In [308]:
df_limpio = df_baseline.copy()

scaler = StandardScaler()
df_limpio["house_size"] = scaler.fit_transform(df_limpio[["house_size"]])
df_limpio["price"] = scaler.fit_transform(df_limpio[["price"]])
df_limpio["SecurityDeposit"] = scaler.fit_transform(df_limpio[["SecurityDeposit"]])
df_limpio["days_since_verif"] = scaler.fit_transform(df_limpio[["days_since_verif"]])
#latitud y longitud tambien se podrian estandarizar


# justificacion de eliminacion 

segun yo seria algo asi: 

isNegotiable = no entrega mucha informacion, solo hay 179 "negotiable", los demas eran todos nan

SecurityDeposit = Es informacion extra que no afecta al precio. #nose este

Currency = Es el tipo de moneda que se utiliza y es siempre el mismo. 

PriceSqFt = No agrega ninguna informacion ya que es siempre 0. 

City = Es la misma ciudad en todos. 


In [309]:
df_limpio.drop(columns=["isNegotiable","currency","priceSqFt","city"], inplace=True)


In [310]:
df_limpio.describe()

,house_size,location,latitude,longitude,price,numBathrooms,numBalconies,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
count,5.000000e+03,5000.000000,5000.000000,5000.000000,5.000000e+03,5000.000000,5000.000000,5.000000e+03,5000.000000,5000.00000,5000.00000,5000.000000,5.000000e+03
mean,-9.094947e-17,136.314400,28.578012,77.174499,3.410605e-17,2.904000,1.069800,-2.273737e-17,1.321200,3.09780,0.02760,1.210400,6.821210e-17
std,1.000100e+00,79.323699,0.190186,0.115636,1.000100e+00,1.104458,1.053731,1.000100e+00,0.715914,1.14162,0.16384,1.045347,1.000100e+00
min,-1.306412e+00,0.000000,20.011379,72.771332,-8.000303e-01,0.000000,0.000000,-6.650176e-01,0.000000,1.00000,0.00000,0.000000,-9.315810e-01
25%,-8.683108e-01,65.000000,28.544489,77.138248,-7.032997e-01,2.000000,0.000000,-6.650176e-01,1.000000,2.00000,0.00000,1.000000,-8.292058e-01
50%,-2.226873e-01,141.000000,28.569295,77.196472,-3.547048e-01,3.000000,1.000000,-6.285733e-01,1.000000,3.00000,0.00000,1.000000,-4.928300e-01
75%,1.343411e+00,195.000000,28.618687,77.228950,2.881044e-01,4.000000,2.000000,3.594829e-01,2.000000,4.00000,0.00000,1.000000,3.846720e-01
max,5.320913e+00,287.000000,28.805466,80.361313,1.017652e+01,10.000000,8.000000,1.087671e+01,2.000000,9.00000,1.00000,5.000000,1.737488e+00


In [311]:
df_interaccion = df_limpio.copy()
df_interaccion["latitude * longitude"] = df_interaccion["latitude"] * df_interaccion["longitude"]

##  Regresion lineal


In [312]:
# Variables
X_baseline = df_baseline.drop(columns=["price"])  # Variables independientes
y_baseline = df_baseline["price"]                 # Variable dependiente

# División del conjunto
X_train, X_test, y_train, y_test = train_test_split(X_baseline, y_baseline, train_size=0.8, random_state=254)
 

In [313]:
# Variables
X_limpio = df_limpio.drop(columns=["price"])  # Variables independientes
y_limpio = df_limpio["price"]                 # Variable dependiente

# División del conjunto
X_train_limpio, X_test_limpio, y_train_limpio, y_test_limpio = train_test_split(X_limpio, y_limpio, train_size=0.8, random_state=254)
 

In [314]:
x_interaccion = df_interaccion.drop(columns=["price"])  # Variables independientes
y_interaccion = df_interaccion["price"]                 # Variable dependiente

# División del conjunto
X_train_interaccion, X_test_interaccion, y_train_interaccion, y_test_interaccion = train_test_split(x_interaccion, y_interaccion, train_size=0.8, random_state=254)


In [315]:
# Ajuste/Entrenamiento del modelo
lr = LinearRegression()
lr.fit(X_train, y_train)
y_prediction = lr.predict(X_test) # y_gorrito

lr.fit(X_train_limpio,y_train_limpio)
y_prediction_limpio = lr.predict(X_test_limpio)

lr.fit(X_train_interaccion,y_train_interaccion)
y_prediction_interaccion = lr.predict(X_test_interaccion)
 

In [316]:
mse = mean_squared_error(y_test, y_prediction)
r2 = r2_score(y_test, y_prediction)

msel = mean_squared_error(y_test_limpio,y_prediction_limpio)
r2l = r2_score(y_test_limpio,y_prediction_limpio)

msei = mean_squared_error(y_test_interaccion,y_prediction_interaccion)
r2i = r2_score(y_test_interaccion,y_prediction_interaccion)

In [317]:
table = PrettyTable()
table.field_names = ["Modelo", "MSE", "R2"]
table.add_row(["Baseline", mse, r2])
table.add_row(["Limpio", msel, r2l]) # Completar
table.add_row(["Limpio + interacción", msei, r2i]) # Completar
table

Modelo,MSE,R2
Baseline,7537504863.499018,0.9115972764946418
Limpio,0.10143735019169517,0.9107104950688836
Limpio + interacción,0.10146030182355562,0.9106902920584312
